# Data

In [1]:
!pip install pyvi
!pip install transformers
!pip install underthesea
!pip install sentence_transformers
!pip install pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.5 MB 35.4 MB/s 
     |████████████████████████████████| 1.0 MB 60.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 28.1 MB/s 
     |████████████████████████████████| 182 kB 80.9 MB/s 
     |████████████████████████████████| 7.6 MB 78.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.0 MB 32.4 MB/s 
     |████████████████████████████████| 591 kB 78.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.3 MB/s 
     |████████████████████████████████| 1.3 MB 80.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3

In [2]:
import json
import torch
import pandas as pd
from tqdm import tqdm
from pyvi import ViTokenizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
import pickle
from sentence_transformers import SentenceTransformer, util
tqdm.pandas()
import json
import re
from transformers import pipeline
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator
from scipy import sparse
import sklearn
import underthesea
import os
from glob import glob
from pprint import pprint
import nltk
from underthesea import word_tokenize
from underthesea import text_normalize
from sklearn.decomposition import TruncatedSVD
import math
import torch
import sklearn
import numpy as np
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
!wget https://dl-challenge.zalo.ai/e2e-question-answering/e2eqa-train+public_test-v1.zip
!wget https://dl-challenge.zalo.ai/e2e-question-answering/wikipedia_20220620_cleaned.zip
!wget https://dl-challenge.zalo.ai/e2e-question-answering/e2eqa-private_test_v3.zip
!wget https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt
!unzip -q /content/e2eqa-train+public_test-v1.zip
!unzip -q /content/wikipedia_20220620_cleaned.zip
!unzip -q /content/e2eqa-private_test_v3.zip

--2022-12-18 12:32:21--  https://dl-challenge.zalo.ai/e2e-question-answering/e2eqa-train+public_test-v1.zip
Resolving dl-challenge.zalo.ai (dl-challenge.zalo.ai)... 49.213.78.231
Connecting to dl-challenge.zalo.ai (dl-challenge.zalo.ai)|49.213.78.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3950125 (3.8M) [application/zip]
Saving to: ‘e2eqa-train+public_test-v1.zip’

e2eqa-train+public_ 100%[===================>]   3.77M  8.23MB/s    in 0.5s    

2022-12-18 12:32:22 (8.23 MB/s) - ‘e2eqa-train+public_test-v1.zip’ saved [3950125/3950125]

--2022-12-18 12:32:22--  https://dl-challenge.zalo.ai/e2e-question-answering/wikipedia_20220620_cleaned.zip
Resolving dl-challenge.zalo.ai (dl-challenge.zalo.ai)... 49.213.78.231
Connecting to dl-challenge.zalo.ai (dl-challenge.zalo.ai)|49.213.78.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 417016977 (398M) [application/zip]
Saving to: ‘wikipedia_20220620_cleaned.zip’

wikipedia_20220620_

In [4]:
data_file = open('e2eqa-train+public_test-v1/zac2022_train_merged_final.json')
data = json.load(data_file)
stopword_file = open('vietnamese-stopwords-dash.txt', 'r')
stopword = stopword_file.read()
stopword = stopword.split('\n')

# ModelForQuestionAnswering

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("nguyenvulebinh/vi-mrc-large")
model = AutoModelForQuestionAnswering.from_pretrained("nguyenvulebinh/vi-mrc-large")

qa_model = pipeline("question-answering", model=model,tokenizer=tokenizer,device=0)


# "hogger32/xlmRoberta-for-VietnameseQA", "nguyenvulebinh/vi-mrc-base"

Downloading:   0%|          | 0.00/398 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [ ]:
def remove_stopword(lsword):
  return lsword
  res = []
  for word in lsword.split():
    if not word in stopword:
      res.append(word) 
  return res

#QA_model

In [ ]:
df = pd.read_json('/content/wikipedia_20220620_cleaned.jsonl', orient='records', lines=True)

In [ ]:
df.drop(columns=['id','url','timestamp','revid'],inplace=True)

In [ ]:
df.to_csv('wiki.csv')

In [ ]:
def normalize_text(text):
    from bs4 import BeautifulSoup
    import re
    def strip_html_tags(text):
        soup = BeautifulSoup(text, "html.parser")
        stripped_text = soup.get_text(separator=" ")
        return stripped_text
    text = strip_html_tags(text)
    # remove 'BULLET::::-'
    text = re.sub(r'BULLET::::-', ' ', text)
    # remove = if more than 1
    text = re.sub(r'={2,}', ' ', text)
    # emove duplicate spaces
    text = re.sub(r'  +', ' ', text)
    # remove duplicate newline characters if more than 2
    text = re.sub(r'\n+', '. ', text)

    # tokenize_text = tokenize(text)
    return text

In [ ]:
df['text'] = df['text'].parallel_apply(normalize_text)
df['text'] = df['text'].parallel_apply(remove_stopword)

In [ ]:
def segment_documents(data,docs, max_doc_length=300):
  # List containing full and segmented docs
  segmented_docs = []
  new_df = pd.DataFrame(columns=['text'])
  i=0
  for doc in tqdm(docs):

    # Split document by spaces to obtain a word count that roughly approximates the token count
    split_to_words = doc.split(" ")

    # If the document is longer than our maximum length, split it up into smaller segments and add them to the list 
    if len(split_to_words) > max_doc_length:
      for doc_segment in range(0, len(split_to_words), max_doc_length):
        a= " ".join(split_to_words[doc_segment:doc_segment + max_doc_length])
        segmented_docs.append(a)
        # temp={'id':data.iloc[i]['id'],'title':data.iloc[i]['title'],'text':a}
        # new_df.loc[len(new_df.index)] = [data.iloc[i]['title'], a]

    # If the document is shorter than our maximum length, add it to the list
    else:
      segmented_docs.append(doc)
      # new_df.loc[len(new_df.index)] = [data.iloc[i]['title'], doc]
      # temp={'id':data.iloc[i]['id'],'title':data.iloc[i]['title'],'text':doc}
      # new_df=new_df.append(temp,ignore_index=True)
    i+=1
    # print('i',i)
    
  return segmented_docs#,new_df
  # return new_df

In [ ]:
sentences=df['text'].tolist()
corpus_split_df=segment_documents(df,sentences)

100%|██████████| 1273469/1273469 [00:18<00:00, 67669.52it/s]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus_split_df)

In [5]:
data_file = open('e2eqa-train+public_test-v1/zac2022_train_merged_final.json')
data = json.load(data_file)

In [ ]:
new_data=[]
for per_dictionary in data['data']:
    if per_dictionary['category']=='FALSE_LONG_ANSWER':
      continue
    new_data.append(per_dictionary)

In [ ]:
# questions_file = open('/content/zac2022_testb_only_question.json')
# questions = json.load(questions_file)

In [ ]:
def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

# def get_gold_answers(example):
#     """helper function that retrieves all possible true answers from a squad2.0 example"""
    
#     gold_answers = [answer["text"] for answer in example.answers if answer["text"]]

#     # if gold_answers doesn't exist it's because this is a negative example - 
#     # the only correct answer is an empty string
#     if not gold_answers:
#         gold_answers = [""]
        
#     return gold_answers

In [ ]:
for q in tqdm(new_data):
    test_question = q['question']
    #Tokenize question in test data
    
    query_vector=vectorizer.transform([test_question])
    sim_maxtrix = sklearn.metrics.pairwise.cosine_similarity(query_vector, X )
    sim_maxtrix = np.reshape(sim_maxtrix, (-1,))
    idx = (-sim_maxtrix).argsort()[:1]
    for _id in idx:
        # print(_id, sim_maxtrix[_id])
        # print(newItems_wiki[_id]['title'].upper())
        # title_retrieval.append(newItems_wiki[_id]['title'])
        # retrieval_result.append(newItems_wiki[_id]['text'])
      result = qa_model(question = test_question, context = corpus_split_df[_id])['answer']
    q['compute_exact_match'] = compute_exact_match(result,q['short_candidate'])
    q['compute_f1'] = compute_f1(result,q['short_candidate'])
    q['predict'] = result